In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
os.chdir('musicautobot')

In [3]:
from musicautobot.numpy_encode import *
from musicautobot.utils.file_processing import process_all, process_file
from musicautobot.config import *
from musicautobot.music_transformer import *
from musicautobot.multitask_transformer import *
from musicautobot.numpy_encode import stream2npenc_parts

In [4]:
from music21 import environment
# environment.set("musescoreDirectPNGPath", "/usr/bin/musescore3")
environment.set("musescoreDirectPNGPath", "/Applications/MuseScore 3.app/Contents/MacOS/mscore")

In [19]:
os.makedirs('../data/numpy/pretrained', exist_ok=True)
os.makedirs('../data/midi', exist_ok=True)

In [20]:
# Config
config = multitask_config();

# Location of your midi files
midi_path =  Path('../data/midi')

# Location of saved datset
data_path = Path('../data/numpy')
data_save_name = 'musicitem_data_save.pkl'

# Data
data = MusicDataBunch.empty(data_path)
vocab = data.vocab

# Download pretrained model if you haven't already
pretrained_url = 'https://ashaw-midi-web-server.s3-us-west-2.amazonaws.com/pretrained/MultitaskLargeKeyC.pth'

pretrained_path = data_path/'pretrained'/Path(pretrained_url).name
download_url(pretrained_url, dest=pretrained_path)



In [21]:
# Learner
learn = multitask_model_learner(data, pretrained_path=pretrained_path)
# learn.to_fp16();

learn.model = learn.model.eval()
learn.model.training = False

In [22]:
from sklearn.decomposition import NMF
from sklearn.preprocessing import MinMaxScaler


In [7]:
import scipy.io as sio
import numpy as np
import re
import pandas as pd
import sklearn.manifold

In [25]:
from os import listdir
from os.path import isfile, join
# mypath = "../suprisial_stimuli/transformer_activation"
onlyfiles = sorted([f for f in listdir(midi_path) if isfile(join(midi_path, f))])

In [157]:
def get_lm_mask(x_len, device='cpu'):
    mask = torch.triu(torch.ones((x_len, x_len), device=device), diagonal=1)[None]
    return mask.bool() if hasattr(mask, 'bool') else mask.byte()

In [156]:
embeddings_byfile = {}

for f in onlyfiles:
    print(f)
    embeddings = []
    item = MusicItem.from_file(os.path.join(midi_path, f), data.vocab)
    x = item.to_tensor().unsqueeze(0)
    x_pos = item.get_pos_tensor().unsqueeze(0)
    
    bs, lm_len = x.size()
    lm_mask = get_lm_mask(lm_len)
    msk_emb = None
            
    lm_emb = learn.model.encoder.embed(x, x_pos)
    if msk_emb is not None and msk_emb.shape[1] > lm_emb.shape[1]:
        pos_enc = learn.model.encoder.embed.relative_pos_enc(msk_emb)
    else:
        pos_enc = learn.model.encoder.embed.relative_pos_enc(lm_emb)
    
    
    
    layer_embeddings = []
    for i, layer in enumerate(learn.model.encoder.layers):
        
        lm_emb = layer(lm_emb, msk_emb, lm_mask=lm_mask,
                    r=pos_enc, g_u=learn.model.encoder.u, g_v=learn.model.encoder.v)
        layer_embeddings.append(lm_emb)
        
    layer_embeddings = torch.stack(layer_embeddings).squeeze(1)

    embeddings_byfile[f] = layer_embeddings
    

audio1.mid
torch.Size([1, 4086, 4086])
audio10.mid
torch.Size([1, 2570, 2570])
audio2.mid
torch.Size([1, 3562, 3562])
audio3.mid
torch.Size([1, 1204, 1204])
audio4.mid
torch.Size([1, 2118, 2118])


/Users/gavin/miniconda3/envs/musicautobot/lib/python3.9/site-packages/music21/midi/translate.py:863: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=5, channel=None, data=b'4th Movement: bour\xe9e'>; getting generic Instrument
  warnings.warn(


audio5.mid
torch.Size([1, 2046, 2046])
audio6.mid
torch.Size([1, 6416, 6416])
audio7.mid
torch.Size([1, 2168, 2168])
audio8.mid
torch.Size([1, 5418, 5418])
audio9.mid
torch.Size([1, 1354, 1354])


In [158]:
for k, v in embeddings_byfile.items():
    print((k, v.shape)) # each is shape (layers, notes, hidden_dimension)

('audio1.mid', torch.Size([12, 4086, 768]))
('audio10.mid', torch.Size([12, 2570, 768]))
('audio2.mid', torch.Size([12, 3562, 768]))
('audio3.mid', torch.Size([12, 1204, 768]))
('audio4.mid', torch.Size([12, 2118, 768]))
('audio5.mid', torch.Size([12, 2046, 768]))
('audio6.mid', torch.Size([12, 6416, 768]))
('audio7.mid', torch.Size([12, 2168, 768]))
('audio8.mid', torch.Size([12, 5418, 768]))
('audio9.mid', torch.Size([12, 1354, 768]))
